#### Common warnings:

1. __Backup your solution into the 'work' directory inside the home directory ('/home/jovyan'). It is the only one that state will be saved over sessions.__

1. Please, ensure that you call the right interpreter (python2 or python3). Do not write just "python" without the major version. There is no guarantee that any particular version of Python is set as the default one in the Grading system.

1. One cell must contain only one programming language.
E.g. if a cell contains Python code and you also want to call a bash-command (using “!”) in it, you should move the bash to another cell.

1. Our IPython converter is an improved version of the standard converter Nbconvert and it can process most of Jupyter's magic commands correctly (e.g. it understands "%%bash" and executes the cell as a "bash"-script). However, we highly recommend to avoid magics wherever possible.

#### Spark specific warnings:

1. It is a good practice to run Spark with master "yarn". However, containered system's performance is limited. If you see repeating Py4JavaErrors or Py4JNetworkErrors exceptions which you assume are not relevant to your code, feel free to change master to “local”.

1. You should eliminate extra symbols in output (such as quotes, brackets etc.). When you finally get the resulting dataframe it is easier to print wiki.take(1) instead of traverse RDD using for cycle. But in this case a lot of junk symbols will be printed like: `[['Anarchism', 'is', .. ]]`. See the right output example in the task.

#### Task hint
Each subsequent of these tasks is a continuation of the previous one. So, you may use the same IPython notebook for all the programming assignments in this week.

In [32]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import Window

spark_session = SparkSession.builder.enableHiveSupport().master("yarn").getOrCreate()

In [33]:
data = spark_session.read.parquet("/data/sample264")
meta = spark_session.read.parquet("/data/meta")

In [34]:
data.show(10)

+------+-------+--------+----------+
|userId|trackId|artistId| timestamp|
+------+-------+--------+----------+
| 13065| 944906|  978428|1501588527|
|101897| 799685|  989262|1501555608|
|215049| 871513|  988199|1501604269|
|309769| 857670|  987809|1501540265|
|397833| 903510|  994595|1501597615|
|501769| 818149|  994975|1501577955|
|601353| 958990|  973098|1501602467|
|710921| 916226|  972031|1501611582|
|  6743| 801006|  994339|1501584964|
|152407| 913509|  994334|1501571055|
+------+-------+--------+----------+
only showing top 10 rows



In [36]:
user_track = data.groupBy('userId', 'trackId').count()

In [37]:
user_track.show()

+------+-------+-----+
|userId|trackId|count|
+------+-------+-----+
|185109| 870292|    4|
| 93053| 915614|    1|
| 55026| 949518|    1|
|640605| 841340|    3|
|103552| 942680|    3|
|227285| 944606|    2|
|105324| 928370|    1|
|647294| 887536|    1|
|324195| 821053|    1|
|513364| 857897|   10|
|712953| 965106|    1|
|497886| 811766|   13|
|211774| 876669|    1|
|663443| 915937|    1|
|313526| 858827|    1|
|275334| 865671|    1|
|154082| 825174|    1|
| 23946| 912683|    1|
|193100| 864299|    1|
|503381| 861413|    2|
+------+-------+-----+
only showing top 20 rows



#### Normalization could be done by next function

In [38]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, sum,col

# normalize weights of its edges (divide the weight of each edge on a sum of weights of all edges).
def norm(df, key1, key2, field, n): 
    
    window = Window.partitionBy(key1).orderBy(col(field).desc())
        
    topsDF = df.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= n) \
        .drop(col("row_number")) 
        
    tmpDF = topsDF.groupBy(col(key1)).agg(col(key1), sum(col(field)).alias("sum_" + field))
   
    normalizedDF = topsDF.join(tmpDF, key1, "inner") \
        .withColumn("norm_" + field, col(field) / col("sum_" + field)) \
        .cache()

    return normalizedDF

In [39]:
user_track_norm = norm(user_track, 'userId', 'trackId', 'count', 1000) \
        .select('userId', 'trackId', 'norm_count')

In [40]:
user_track_norm.take(50)

[Row(userId=3175, trackId=947718, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=940951, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=845631, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=864690, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=831005, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=930432, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=965012, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=858940, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=829307, norm_count=0.1111111111111111),
 Row(userId=5518, trackId=961148, norm_count=0.5),
 Row(userId=5518, trackId=873588, norm_count=0.3333333333333333),
 Row(userId=5518, trackId=930964, norm_count=0.16666666666666666),
 Row(userId=5803, trackId=810419, norm_count=1.0),
 Row(userId=6654, trackId=802183, norm_count=0.2),
 Row(userId=6654, trackId=886091, norm_count=0.2),
 Row(userId=6654, trackId=825094, norm_count=0.2),
 Row(userId=6654, 

In [41]:
window = Window.orderBy(f.col('norm_count').desc())
    
user_TrackList = user_track_norm.withColumn('position', f.rank().over(window)) \
    .filter(f.col('position') < 40) \
    .orderBy('userId', 'trackId') \
    .select('userId', 'trackId') \
    .take(40)

In [42]:
for val in user_TrackList:
    print("%s %s" % val)

66 965774
116 867268
128 852564
131 880170
195 946408
215 860111
235 897176
300 857973
321 915545
328 943482
333 818202
346 864911
356 961308
428 943572
431 902497
445 831381
488 841340
542 815388
617 946395
649 901672
658 937522
662 881433
698 935934
708 952432
746 879259
747 879259
776 946408
784 806468
806 866581
811 948017
837 799685
901 871513
923 879322
934 940714
957 945183
989 878364
999 967768
1006 962774
1049 849484
1057 920458
